## load modules

In [1]:
from numpy import genfromtxt
import numpy as np
import math
from pylab import *
import urllib
import os
import matplotlib.pyplot as pl
import pandas as pd
from sklearn.metrics import mean_squared_error
from __future__ import division
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from lmfit import Model
import pickle
pl.style.use('seaborn')
pl.rc('font',family='Arial')

### constants


In [2]:
temp = 300.
Ms = 420000.
kb = 1.381e-23 
tau0 = 1e-10
mu0 = 4*np.pi*1e-7

In [3]:
def visc_calc(T, mass_fraction):
    glycerolDen = (1273.3-0.6121*T)/1000 			#Density of Glycerol (g/cm3)
    waterDen = (1-math.pow(((abs(T-4))/622),1.7)) 	#Density of water (g/cm3)
    
    ##Andreas Volk polynomial method
    contraction_av = 1-math.pow(3.520E-8*((mass_fraction*100)),3)+math.pow(1.027E-6*((mass_fraction*100)),2)+2.5E-4*(mass_fraction*100)-1.691E-4
    contraction = 1+contraction_av/100
    
    glycerolVisc=0.001*12100*np.exp((-1233+T)*T/(9900+70*T))
    waterVisc=0.001*1.790*np.exp((-1230-T)*T/(36100+360*T))
    
    a=0.705-0.0017*T
    b=(4.9+0.036*T)*np.power(a,2.5)
    alpha=1-mass_fraction+(a*b*mass_fraction*(1-mass_fraction))/(a*mass_fraction+b*(1-mass_fraction))
    A=np.log(waterVisc/glycerolVisc)
    
    viscosity_mix=glycerolVisc*np.exp(A*alpha)
    
    return viscosity_mix

def power(my_list, p):
    return [ x**p for x in my_list ]

def calc_neel(size, k):
    neel = []
    for x in range(len(size)):
        neel.append(tau0*np.exp(k[x]*4*np.pi*(1e-9*float(size[x]))**3*(24*kb*temp)**(-1)))
    return neel

def calc_brown(eta, hsize):
    brown = []
    for x in range(len(eta)):
        brown.append(eta[x]*np.pi*(hsize[x]*1e-9)**3*(2*kb*temp)**(-1))
    return brown

def vol(my_list):
    return[ np.pi*(1e-9*x)**3/6. for x in my_list ]


## import simulation data

In [4]:
freqs = []
fields = []
size = []
third = []
fifth = []
hsize = []
sigma = []
eta = []
b = []
K = []
phase1 = []
phase3 = []
gyro = []

for f in os.listdir('fits'):
    if f[-1] == 'v':
        filename = 'fits/' + f
        filedata = np.genfromtxt(filename, delimiter=',')
        f = f[:-4]
        s = str.split(f,"_")
        size.append(float(s[0][:-2]))
        fields.append(float(s[1][:-2]))
        freqs.append(int(s[2][:-3])*1000)
        sigma.append(0.1)
        eta.append(8.9e-4)
        moment = filedata[:,1]
        third.append(abs(np.fft.fft(moment)[3]))
        fifth.append(abs(np.fft.fft(moment)[5]))
        phase1.append(np.angle(np.fft.fft(moment)[1]))
        phase3.append(np.angle(np.fft.fft(moment)[3]))
        if s[-1] == "frozen":
            b.append("off")
        else:
            b.append("on")     
        g = 1.
        h = 50.
        k = 5000.
        for ss in range(2,len(s)):
            if s[ss][0] == "g":
                g = float(s[ss][1:])
            if s[ss][0] == "K":
                k = 1000*float(s[ss][1:])
                if k > 25000:
                    k /= 1000.
            if s[ss][0] == "H":
                h = float(s[ss][1:])
        K.append(k)
        hsize.append(h)
        gyro.append(g)
            
sdata = pd.DataFrame({'frequency': freqs, 'field': fields,'size': size, 'sigma': sigma, \
                     'third': third, 'fifth': fifth, 'hsize': hsize, 'eta': eta, 'b': b, \
                     'K': K, 'phase1': phase1, 'phase3': phase3, 'gyro': gyro})

sdata['5:3'] = sdata['fifth']/sdata['third']
sdata['angF'] = 2*np.pi*sdata['frequency']
sdata['Vc'] = vol(sdata['size'])
sdata['Ms'] = 393023 * (1 - np.exp(-2.78258e8 * sdata['size']*1e-9))**57.87571 
sdata['tauB'] = calc_brown(sdata['eta'],sdata['hsize'])
sdata['tauN'] = calc_neel(sdata['size'], sdata['K'])
sdata['tan1'] = abs(np.tan(sdata['phase1']))
sdata['tan3'] = abs(np.tan(sdata['phase3']))
sdata['tau'] = sdata['tan1']/sdata['angF']

for index, row in sdata.iterrows():
    T = row['frequency']**(-1)
    t = np.arange(0,T,T/1000.)
    H = 0.001*row['field']*np.cos(row['angF']*t)
    V = (4/3)*np.pi*(row['size']*1e-9/2)**3
    xi = row['Ms']*V*H/(kb*temp)
    L = np.cosh(xi)/np.sinh(xi) - 1/xi
    sdata.loc[index,'L5:3'] = abs(np.fft.fft(L)[5])/abs(np.fft.fft(L)[3])

## train model

### Brownian rotation off (frozen)

In [30]:
mldataF = sdata[(sdata['b'] == 'off') & (sdata['field'] >9) & (sdata['size'] > 12)]
X = mldataF[['field','frequency','size','K', 'gyro']]
Y1 = mldataF[['tan1']]
Y2 = mldataF[['tau']]*1e9  #tau in nanoseconds
Y3 = mldataF[['5:3']]
Y4 = mldataF[['fifth']]
Y5 = mldataF[['third']]

X_train, X_test , Y1_train , Y1_test = train_test_split(X,Y1,test_size=0.15,random_state=26)
model1 = RandomForestRegressor(n_estimators=100, max_features=5)
model1.fit(X_train, Y1_train.values.ravel())

X_train, X_test , Y2_train , Y2_test = train_test_split(X,Y2,test_size=0.15,random_state=26)
model2 = RandomForestRegressor(n_estimators=100, max_features=5)
model2.fit(X_train, Y2_train.values.ravel())

X_train, X_test , Y3_train , Y3_test = train_test_split(X,Y3,test_size=0.15,random_state=26)
model3 = RandomForestRegressor(n_estimators=100, max_features=5)
model3.fit(X_train, Y3_train.values.ravel())

X_train, X_test , Y4_train , Y4_test = train_test_split(X,Y4,test_size=0.15,random_state=26)
model4 = RandomForestRegressor(n_estimators=100, max_features=5)
model4.fit(X_train, Y4_train.values.ravel())

X_train, X_test , Y5_train , Y6_test = train_test_split(X,Y5,test_size=0.15,random_state=26)
model5 = RandomForestRegressor(n_estimators=100, max_features=5)
model5.fit(X_train, Y5_train.values.ravel())

print(model1.score(X_train,Y1_train))
print(model2.score(X_train,Y2_train))
print(model3.score(X_train,Y3_train))
print(model4.score(X_train,Y4_train))
print(model5.score(X_train,Y5_train))

0.999848354684
0.99953762244
0.995162020159
0.999905418011
0.999907008526


### output model

In [32]:
output1 = open('mpsmodel-tan-frozen.pkl', 'wb')
pickle.dump(model1, output1)
output1.close()

output2 = open('mpsmodel-tau-frozen.pkl', 'wb')
pickle.dump(model2, output2)
output2.close()

output3 = open('mpsmodel-ratio-frozen.pkl', 'wb')
pickle.dump(model3, output3)
output3.close()

output4 = open('mpsmodel-fifth-frozen.pkl', 'wb')
pickle.dump(model4, output4)
output4.close()

output5 = open('mpsmodel-third-frozen.pkl', 'wb')
pickle.dump(model5, output5)
output5.close()

### Brownian rotation on

In [33]:
mldata = sdata[(sdata['b'] == 'on') & (sdata['field'] >9) & (sdata['size'] > 12)]
X = mldata[['field','frequency','size','K', 'gyro','eta']]
Y1 = mldata[['tan1']]
Y2 = mldata[['tau']]*1e9  #tau in nanoseconds
Y3 = mldata[['5:3']]
Y4 = mldata[['fifth']]
Y5 = mldata[['third']]

X_train, X_test , Y1_train , Y1_test = train_test_split(X,Y1,test_size=0.15,random_state=26)
model1b = RandomForestRegressor(n_estimators=100, max_features=6)
model1b.fit(X_train, Y1_train.values.ravel())

X_train, X_test , Y2_train , Y2_test = train_test_split(X,Y2,test_size=0.15,random_state=26)
model2b = RandomForestRegressor(n_estimators=100, max_features=6)
model2b.fit(X_train, Y2_train.values.ravel())

X_train, X_test , Y3_train , Y3_test = train_test_split(X,Y3,test_size=0.15,random_state=26)
model3b = RandomForestRegressor(n_estimators=100, max_features=6)
model3b.fit(X_train, Y3_train.values.ravel())

X_train, X_test , Y4_train , Y4_test = train_test_split(X,Y4,test_size=0.15,random_state=26)
model4b = RandomForestRegressor(n_estimators=100, max_features=6)
model4b.fit(X_train, Y4_train.values.ravel())

X_train, X_test , Y5_train , Y6_test = train_test_split(X,Y5,test_size=0.15,random_state=26)
model5b = RandomForestRegressor(n_estimators=100, max_features=6)
model5b.fit(X_train, Y5_train.values.ravel())

print(model1b.score(X_train,Y1_train))
print(model2b.score(X_train,Y2_train))
print(model3b.score(X_train,Y3_train))
print(model4b.score(X_train,Y4_train))
print(model5b.score(X_train,Y5_train))

0.998602140916
0.999349246132
0.990852441302
0.998147928533
0.998750912384


### output model

In [35]:
output1b = open('mpsmodel-tan.pkl', 'wb')
pickle.dump(model1b, output1b)
output1b.close()

output2b = open('mpsmodel-tau.pkl', 'wb')
pickle.dump(model2b, output2b)
output2b.close()

output3b = open('mpsmodel-ratio.pkl', 'wb')
pickle.dump(model3b, output3b)
output3b.close()

output4b = open('mpsmodel-fifth.pkl', 'wb')
pickle.dump(model4b, output4b)
output4b.close()

output5b = open('mpsmodel-third.pkl', 'wb')
pickle.dump(model5b, output5b)
output5b.close()